In [ ]:
#work in progress - this assumes spark installed and does not point to sparkge folder
import sparkge
from sparkge.x.stores import aerospike_store
from sparkge.x.fitness import regressor_fitness
#grammar
with context(store=x.as_store(),start_symbol=expr) as c: 
    c.run(eval_func=regressor_fitness(c.store.lazy_query("ASM_DATA")) )

In [ ]:
class store:
    def __init__(self,options):
        pass
    
    def update(self,matrix, options):
        pass
    
    def broadcast(self):
        pass
    
    def report(self):
        pass
    
    def lazy_query(self,qry_str):
        def _query():
            pass
        return _query
    
class options(dict):
    def __init__(self):pass
    
class context:
    def __init__(self,store_provider,start_symbol,options=None):
        self.store = store_provider
        self.options = options
        self.start_symbol = start_symbol
        self.sc = None# setup spark context in this context
    
    def __init__(self):
        pass
    def __enter__(self):
        self.store.broadcast()
    def __exit__(self):
        self.store.report()        
    def __iter__(self):
        return iter(store) 
    def __getitem__(self,key):
        return store[key]   
    @property
    def keys(self):
        pass
 
    def transcribe(self, genome, params = []):
        phenotype = self.start_symbol(genome,params)
        return accepting_ordered_params(phenotype,params)#think about who controls the param order
    
    def evolve(self,options):
        self.options.mutator.mutate(self.store)
        sel = self.options.selector.best_keys(self.store)
        self.options.combiner.combine(sel, self.store)
        
    def run(self,eval_func):   
        #transcribe returns function of a matrix and eval_func_prov takes such a thing
        worker_fun = lambda key: eval_func(self.transcribe(self.cache[key]))
        for gen in self.options.generations:  
            results = self.sc.map(self.keys).reduce(lambda key: worker_fun(key)).collect()
            self.store.update(results)
            self.evolve()
        
def xnorm(Y,Y_dash):  pass

def regressor_fitness(mat_query_func,options,norm=xnorm):
    def _fitness(f_of_mat):
        mat = mat_query_func()
        X = mat[:,0:-1]
        Y = mat[:,-1]
        Y_dash = f_of_mat(X) 
        return norm(Y,Y_dash)
    return _fitness

with sparkge.context(store=x.as_store(),
                     start_symbol=expr,#x.load_symbol("asm_reg.bnf"),
                     options={}) as c: 
    c.run( eval_func= regressor_fitness(store.lazy_query("ASM_DATA")) )

In [1]:
from functools import wraps
from operator import *
from math import *
import random

#make mapping exception type here with some info possibly

#add to evo
class chain:
    
    def __init__(self,l=None, max_wraps=3, default_length=30): 
        if l is None: l = np.random.randint(0,10,default_length)
        l = np.concatenate([l[~np.isnan(l)].astype(int) for i in range(max_wraps)])
        self._l, self.length = l,len(l)
        self._index = 0
     
    def __getitem__(self, key):
        if key >= self.length: return None
        return self._l[ key ]
    
    def _next(self): 
        if self._l is None: return random.randint(0,10)
        if self._index < self.length: 
            self._index += 1
            return self[self._index-1]
        return None
        
    def choose(self, choices):
        cn = self._next()
        if cn ==None: raise MappingException("mapping exception")
        return choices[cn % len(choices)] if len(choices) > 1 else choices[0]

#todo chain.choose(list) -> returns None when chain is empty and mapping should be terminated
#info = [], F = var(chain(),info) -> call F(X=, Y=)
def terminal(original_function=None, is_constant=False):
    def _decorate(function):      
        choices = function()  
        def _terminal(genome,info=None):
            v = genome.choose(choices)
            def __terminal(**kwargs):
                if is_constant==False and info != None and v not in info:info.append(v)
                return kwargs[v]
            return __terminal
        return _terminal
    if original_function: return _decorate(original_function)
    return _decorate

#B = bop(exp,exp),B=B(c),B(X=1)
def operator(original_function=None):
    def _decorate(function):      
        choices = function()  
        def _op(*exp_args):          #f(exp,exp)->operator of cardinality
            def __op(genome,info=None):         #f(chain)->expression 
                args = [f(genome,info) for f in exp_args] 
                op = genome.choose(choices)
                def ___op(**kwargs): #f(X,a,b,c,d)
                    invoked = [f(**kwargs) for f in args]  
                    return op(*invoked)
                return ___op
            return __op
        return _op
    if original_function: return _decorate(original_function)
    return _decorate

def accepting_ordered_params(f,params):
    def _f(*args):
        ordered_args = dict(zip(params,args))
        return f(**ordered_args)
    return _f      

class MappingException(Exception):
    pass

# def expr(genome=None, args = []):
#     try: 
#         f = genome.choose([op(expr,expr), var])
#         expressed = f(genome,args)
#         def _f(**kwargs): return expressed(**kwargs)
#         return _f
#     except MappingException as mex: 
#         def _f(**kwargs): return np.nan
#         return _f

#abstract this faff        
def as_expression(genome, choices, args=[]):
    try: 
        f  = genome.choose(choices)
        expressed = f(genome,choices)
        def _f(**kwargs): return expressed(**kwargs)
        return _f
    except MappingException as mex: 
        def _f(**kwargs): return np.nan
        return _f
    

In [15]:
@terminal
def var():return ["X"]

@operator()
def op(): return [add, sub, mul]

def expr(genome=None, args = []): 
    choices = [op(expr,expr), var]
    return as_expression(genome, choices,args)



In [16]:
import numpy as np
c = chain()
E = expr(c)
E(X=5,Y=10)

5

In [ ]:
accepting_ordered_params(E,args)(5)

In [ ]:
def _get_dict_(F) : return dict(zip(F.__code__.co_freevars, (c.cell_contents for c in F.__closure__)))

def _repr_(f):
    d = _get_dict_(f)
    if "expressed" in list(d.keys()):  return "{}".format(_repr_(d["expressed"]))
    elif "op" in list(d.keys()):
        op = d["op"].__name__
        if op == "sub":  return "({}-{})".format(_repr_(d["args"][0]),  _repr_(d["args"][1]))
        if op == "add":  return "({}+{})".format(_repr_(d["args"][0]),  _repr_(d["args"][1]))
        if op == "mul":  return "({}*{})".format(_repr_(d["args"][0]),  _repr_(d["args"][1]))
        else:  return "({}/{})".format(_repr_(d["args"][0]),  _repr_(d["args"][1]))
    if "f" in d: return _repr_(d["f"])
    else:   return d["v"]

In [ ]:
from sympy import sympify

print(_repr_(E))
sympify(_repr_(E))

# Populace Matrices

In [ ]:
#strategy trait

#this is a lazy mutation matrix - we do not try to find distinct mutation sites because it is inefficient
#sometimes it might be important to be precise about mutation rate but here the rate is a proxy for the true rate
#The cut matrix can be chosen more cleverly using a distribution centered around the mean individual length
%matplotlib inline
import numpy as np
np.seterr(divide='ignore', invalid='ignore') #be careful
import pandas as pd
GENERATIONS = 500
CODON_MAX  = 100

#put these as strategy parameters 
MIN_L, MAX_L, N = 10,100,1000
MRATE = 0.01
RATE = 0.51

#evo helpers global
lpad = lambda v,n : np.pad(v.astype(float), (n-len(v),0), 'constant', constant_values=np.NAN)
rpad = lambda v,n : np.pad(v.astype(float), (0,n-len(v)), 'constant', constant_values=np.NAN)
random_individual = lambda l : np.random.randint(0,CODON_MAX, size=l)
random_padded_individual = lambda l,L : rpad(random_individual(l),L)
lengths = lambda p : np.sum(~np.isnan(p),axis=1)

#promote to functions
generate_from_lengths = lambda ls : np.stack( [random_padded_individual(l,MAX_L) for l in ls])
cut_matrix = lambda n :np.stack([rpad(np.ones(l),MAX_L) for l in __cuts__(n)])

def __cuts__(n,lamda=MAX_L/2):
    """normally distributed about half the average length"""
    return np.abs(np.random.normal(lamda/2,np.sqrt(MAX_L/2),n).astype(int))

def init():
    individual_lengths = np.random.randint(MIN_L, MAX_L, size=N)
    individual_fitnesses = np.full(N,np.nan)
    individual_genotypes = generate_from_lengths(individual_lengths)
    return individual_lengths, individual_fitnesses, individual_genotypes

def _x(a,b):
    c = np.concatenate([a,b],axis=1)
    def f(row):
        r = row[~np.isnan(row)]
        return rpad(r, MAX_L) if len(r) < MAX_L else r[:MAX_L]
    return np.apply_along_axis(f, 1, c)
    
def X(pool):
    chromosomes = pool * cut_matrix(len(pool))
    split = int(len(pool) / 2) # it should be known that this is div by 2
    return np.concatenate([_x(chromosomes[split:], chromosomes[:split]),
                          _x(chromosomes[:split], chromosomes[split:])])

def selection(individuals):
    K = int(RATE*N) 
    mates = 2*int(N-K)
    replacement = N-mates
    scores = individuals[:,1] #1 is the index of scores on the ind matrix
    scores = scores - scores.max() #minimise
    scores = scores / scores.sum() #normalize
    selected = np.random.choice(range(N),mates+replacement,p=scores) 
    #maybe take some random new ones i.e. freaks from nowhere
    return selected[:mates], selected[mates:]

def mutations(rate):
    mar = np.zeros(N*MAX_L)
    idx = np.random.choice(range(N*MAX_L),int(N*MAX_L*rate),replace=False) #np.random.randint(N*MAX_L, size=int(N*MAX_L*MRATE))
    mar[idx]= np.random.randint(CODON_MAX, size=len(idx))
    return mar.reshape((N,MAX_L))

#test - expect normal distribution around lengths - end of story - tune for this
#test-select for large genotypes or sum might do some-ish and allow mut
#two criteria - normal distribution around "good" plus diversity in the geontypes
def evaluate(genotypes): return np.random.uniform(10,100, len(genotypes)) # 
def evaluate(genotypes): return lengths(genotypes) * 2 
def evaluate(genotypes): return  np.apply_along_axis(lambda row : len(row[row%2!=0]), 1,  individual_genotypes) #select for?
#convention - eval returns a distance from perfect - we are trying to minimise

In [ ]:
#real fitness - select for something that can generate X^4 + x^3 + x^ 2 + x^1


In [ ]:
def _evaluate(population, target_function= lambda x : 2*x*x- x**3+x**4, sample_x=list(range(4,7)), return_all=False):
    #move thes out?
    def __express_symbol__(x):
        def _f_(row): return expr(chain(row))(X=x)
        return _f_
    
    def __print_symbol__(row):
        try: return repr(sympify(_repr_(expr(chain(row)))))
        except: return None
    
    df_res = pd.DataFrame(index=range(len(population)))
    df_res["type"] = "result"
    df_tar = pd.DataFrame(index=range(len(population)))
    df_tar["type"] = "target"
    
    for x in sample_x: df_res[str(x)] = np.apply_along_axis(__express_symbol__(float(x)), 1, population)
    for x in sample_x: df_tar[str(x)] = target_function(x)
        
    df = pd.concat([df_res,df_tar])
    df= df.pivot(df.index, "type")
    df.columns = df.columns.swaplevel(0, 1)
    df.sortlevel(0, axis=1, inplace=True)
    df["error"] = np.abs(df["result"].fillna(np.inf) - df["target"]).sum(axis=1)
    #df["repr"] = np.apply_along_axis(__print_symbol__, 1, individual_genotypes)
    if return_all: return df
    return df["error"].values


In [ ]:
individual_lengths, individual_fitnesses, individual_genotypes = init()
#individual_genotypes

# Usage

In [ ]:
_lengths = []
min_error = []
goods = []
for i, generation in enumerate(range(GENERATIONS)): 
    individual_fitnesses = _evaluate(individual_genotypes) #optimize
    individuals = np.stack([lengths(individual_genotypes),individual_fitnesses],axis=1)
    
    if individual_fitnesses.min() ==0: 
        examplar = individual_genotypes[np.argmin(individual_fitnesses)]
        print(repr(sympify(_repr_(expr(chain(examplar))))), "found")
        goods.append(examplar)
        
    pool_index,replacement_index = selection(individuals)
    individual_genotypes = np.concatenate([individual_genotypes[replacement_index], 
                                         X(individual_genotypes[pool_index])]) + mutations(0.01)
    
    _lengths.append(lengths(individual_genotypes).mean())
    min_error.append(individual_fitnesses.min())
    
pd.DataFrame(_lengths).hist()
#geontype distribution around the "fitness selectors"
individual_genotypes

In [ ]:
pd.DataFrame(min_error).plot(figsize=(20,10))

In [ ]:

repr(sympify(_repr_(expr(chain(goods[1])))))

In [ ]:
repr(sympify(_repr_(expr(chain(goods[0])))))

In [ ]:
#_evaluate(individual_genotypes,return_all=True).sort_values("error")

In [ ]:
#if selecting for a we can get the ratio
a = len(individual_genotypes[(individual_genotypes % 2 ==0) & (~np.isnan(individual_genotypes))])
b= len(individual_genotypes[(individual_genotypes % 2 != 0) & (~np.isnan(individual_genotypes))])
b/(a+b) #are odd

In [ ]:
#codon distribution
pd.DataFrame(individual_genotypes.reshape(N*MAX_L)).hist(bins=100)

In [ ]:
pd.DataFrame(individual_genotypes)

In [ ]:
# Test mappings
individual_genotypes

In [ ]:
args = []
expr(chain(individual_genotypes[1]))(X=10)

In [ ]:
results = evaluate(individual_genotypes).sort_values("error")
# results = results.style.set_properties(subset=['repr'], **{'width': '300px'})
# pd.set_option('display.max_colwidth', -1)
results